In [1]:
import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rx
import itertools
import model_afolu as mafl
import owslib
from owslib.wcs import WebCoverageService
from soilgrids import SoilGrids
import pyproj

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:931: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


In [2]:
dir_data = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/AFOLU"

# set names 
# source of tif: https://files.isric.org/soilgrids/latest/data_aggregated/1000m/ocs/
fn_soils = "ocs_0-30cm_mean_1000.tif"
fn_countries = "WB_countries_Admin0_10m"

fp_soils = os.path.join(dir_data, fn_soils)
fp_countries = os.path.join(dir_data, fn_countries, f"{fn_countries}.shp")

model_afolu = mafl.AFOLU(sa.model_attributes)


# get some geo data: 
# WKT for SoilGrids from https://www.isric.org/explore/soilgrids/faq-soilgrids
#
wkt_soil_grid = """
    PROJCS["Homolosine", 
    GEOGCS["WGS 84", 
        DATUM["WGS_1984", 
            SPHEROID["WGS 84",6378137,298.257223563, 
                AUTHORITY["EPSG","7030"]], 
   AUTHORITY["EPSG","6326"]], 
        PRIMEM["Greenwich",0, 
            AUTHORITY["EPSG","8901"]], 
        UNIT["degree",0.0174532925199433, 
            AUTHORITY["EPSG","9122"]], 
        AUTHORITY["EPSG","4326"]], 
    PROJECTION["Interrupted_Goode_Homolosine"], 
    UNIT["Meter",1]]
"""
#
# information on reprojecting using WKT: https://pyproj4.github.io/pyproj/dev/gotchas.html
#
wkt_soil_grid = wkt_soil_grid.replace("\n", "").replace("    ", "")
crs_base = pyproj.CRS.from_epsg(4326)
crs_soil = pyproj.CRS.from_wkt(wkt_soil_grid)
tform = pyproj.Transformer.from_crs(crs_base,crs_soil)

In [15]:
# convert geotiff to dataframe
rx_array = rx.open_rasterio(fp_soils)
df_soils = rx_array[0].to_pandas()


In [17]:
key_ocs = "ocs_0-30cm_mean"

# see rough bounds: http://bboxfinder.com/#-55.776573,-126.386719,33.431441,-33.398438
x_max = -34
x_min = -119
y_max = 33
y_min = -58
# ymin, xmin, ymax, xmax 

"""
soil_grids = SoilGrids()
response = soil_grids.get_coverage_data(
    service_id = "ocs", 
    coverage_id = "ocs_0-30cm_mean",
    west = min_x,
    south = min_y, 
    east = max_x, 
    north = max_y,
    crs = "urn:ogc:def:crs:EPSG::28992",
    output = "output.tif"
)
"""
pts = [
    (x_min, y_min),
    (x_min, y_max),
    (x_max, y_max),
    (x_max, y_min)
]

# create points in soil grids projection
gdf_test = gpd.GeoDataFrame(
    range(1, len(pts) + 1), 
    geometry = gpd.points_from_xy([x[0] for x in pts], [x[1] for x in pts])
)
gdf_test.crs = crs_base
gdf_test = gdf_test.to_crs(crs_soil)
bds = gdf_test.bounds#["minx"]
min_x = min(bds["minx"])
min_y = min(bds["miny"])
max_x = max(bds["maxx"])
max_y = max(bds["maxy"])


flag_empty = -32768
inds_all = list(df_soils.index)
inds_keep = [x for x in inds_all if (x <= max_y) and (x >= min_y)]
inds_keep = [inds_all.index(x) for x in inds_keep]
fields_keep = [x for x in df_soils.columns if (x <= max_x) and (x >= min_x)]

df_soils_red = df_soils[fields_keep].iloc[inds_keep]

# drop columns
drops = []
for k in df_soils_red.columns:
    if len(set(df_soils_red[k])) == 1:
        drops.append(k)

# rows to drop
drops_row = []
for i in range(len(df_soils_red)):
    s = set(df_soils_red.iloc[i])
    if (len(s) == 1) and (flag_empty in s):
        drops_row.append(i)


df_soils_red.drop(drops, axis = 1, inplace = True)
df_soils_red.drop(df_soils_red.index[drops_row], axis = 0, inplace = True)
df_soils_red.reset_index(inplace = True)
# reshape
dfs = pd.melt(df_soils_red, ["y"]);
dfs = dfs[dfs["value"] != flag_empty].reset_index(drop = True)

# create points with soils
gdf_soils = gpd.GeoDataFrame(
    dfs, 
    geometry = gpd.points_from_xy(dfs["x"], dfs["y"])
)
gdf_soils.crs = crs_soil
# reproject
print("reprojecting...")
gdf_soils = gdf_soils.to_crs(crs_base)


reprojecting...


In [304]:
gdf_soils = gdf_soils.to_crs(crs_base)

# update x/y
gdf_soils["x"] = gdf_soils["geometry"].x
gdf_soils["y"] = gdf_soils["geometry"].y

In [308]:
gdf_soils.to_file(os.path.join(dir_data, "soil_grids_lac.shp"))

In [6]:
gdf_soils = gpd.read_file(os.path.join(dir_data, "soil_grids_lac_OSC_30cm", "soil_grids_lac.shp"))

In [ ]:
gdf_soils = 0

In [19]:
# get countries 
countries_keep = list(sa.model_attributes.dict_attributes.get("region").table["category_name"])
gdf_world = gpd.read_file(fp_countries)

# some replacements
field_en = "NAME_EN"
dict_repl = {"The Bahamas": "Bahamas"}
gdf_world[field_en] = gdf_world[field_en].replace(dict_repl)
gdf_world_red = gdf_world[gdf_world[field_en].isin(countries_keep)]
gdf_lac = gdf_world_red[["NAME_EN", "geometry"]]
gdf_lac_bounds = gdf_lac.bounds

In [38]:

i = 0
sep = "---"*25
print("Copying soil gdf...\n\n")
gdf_soils_cut = gdf_soils.copy()


df_out = []
df_agg_out = []
t0 = time.time()
list_times = [t0]
n = len(gdf_lac_bounds)
field_value = "scc"
agg_func = "mean"
while i < n:
    
    # country
    gdf_country = gdf_lac.iloc[i:(i + 1)]
    bounds = gdf_lac_bounds.iloc[i:(i + 1)]

    country = str(gdf_country["NAME_EN"].iloc[0])
    maxx = float(bounds["maxx"].iloc[0])
    minx = float(bounds["minx"].iloc[0])
    maxy = float(bounds["maxy"].iloc[0])
    miny = float(bounds["miny"].iloc[0])
    
    pos = i + 1
    print(f"Starting country {country}\n{sep} ({pos}/{n})\n")
    
    # get points
    print("Filtering points...\n")
    gdf_points = gdf_soils_cut[
        (gdf_soils_cut["x"] <= maxx) &
        (gdf_soils_cut["x"] >= minx) &
        (gdf_soils_cut["y"] <= maxy) &
        (gdf_soils_cut["y"] >= miny)
    ]
    
    print(f"Merging points within bbox x = ({minx}, {maxx}), y = ({miny}, {maxy}) to country...\n")
    gdf_join = gpd.sjoin(
        gdf_country, 
        gdf_points[["value", "geometry"]].rename(columns = {"value": field_value}),
        how = "inner"
    ).drop("geometry", axis = 1)
    
    
    print(f"Getting average soil carbon content by country...\n")
    gdf_agg = gdf_join[["NAME_EN", field_value]].copy()
    gdf_agg = gdf_agg[["NAME_EN", field_value]].groupby(["NAME_EN"]).agg(
        {"NAME_EN": "first", field_value: agg_func}
    ).reset_index(drop = True)

    
    print("Appending output...")
    df_out.append(gdf_join)
    df_agg_out.append(gdf_agg)
    
    
    print("Dropping indices...")
    indices_to_drop = list(gdf_join["index_right"])
    gdf_soils_cut.drop(indices_to_drop, axis = 0, inplace = True)
    
    
    i += 1
    list_times.append(time.time())
    t_delta = np.round(list_times[i] - list_times[i - 1], 2)
    t_elapsed = np.round(list_times[i] - list_times[0])
    
    print(f"\n{sep}\n\nCountry {country} complete in {t_delta} seconds ({t_elapsed} seconds overall)\n\n{sep}\n\n\n")
    

Copying soil gdf...


Starting country Chile
--------------------------------------------------------------------------- (1/26)

Filtering points...

Merging points within bbox x = (-109.45372473890552, -66.42080644356247), y = (-55.918504222591764, -17.50658819768711) to country...

Getting average soil carbon content by country...

Appending output...
Dropping indices...

---------------------------------------------------------------------------

Country Chile complete in 59.03 seconds (59.0 seconds overall)

---------------------------------------------------------------------------



Starting country Bolivia
--------------------------------------------------------------------------- (2/26)

Filtering points...

Merging points within bbox x = (-69.66649226960254, -57.46566076733052), y = (-22.897257587567594, -9.679821471783441) to country...

Getting average soil carbon content by country...

Appending output...
Dropping indices...

-----------------------------------------------

In [47]:
# write output 
df_agg = pd.concat(df_agg_out, axis = 0).sort_values(
    by = ["NAME_EN"]
).reset_index(
    drop = True
)

df_total = pd.concat(df_out, axis = 0).sort_values(
    by = ["NAME_EN"]
).reset_index(
    drop = True
)

df_agg.to_csv(sa.fp_csv_soc_average_soc_by_country, index = None, encoding = "UTF-8")
df_total.to_csv(sa.fp_csv_soc_cells_merged_to_country, index = None, encoding = "UTF-8")



In [58]:
##  CREATE FINAL TABLE


# get vars to update
varlist_soc = []
# all SOC vars
varlist_soc += sa.model_attributes.build_varlist(sa.model_attributes.subsec_name_soil, model_afolu.modvar_soil_organic_c_stocks)

def clean_countries(country):
    return country.strip().replace(" ", "_").lower()

# setup
df_agg_all_fields = df_agg.copy()
field_country = "country"
field_cp = "scc"
# dummy merge
merge_val = -1
field_merge = "merge"

for var in varlist_soc:
    df_agg_all_fields[var] = np.array(df_agg_all_fields[field_cp])

df_agg_all_fields.drop(field_cp, axis = 1, inplace = True)
df_agg_all_fields.rename(columns = {"NAME_EN": field_country}, inplace = True)
df_agg_all_fields[field_country] = df_agg_all_fields[field_country].apply(clean_countries)
df_agg_all_fields[field_merge] = merge_val

attr_tp = sa.model_attributes.dict_attributes.get("dim_time_period")
df_tp = attr_tp.table[[attr_tp.key]].copy()
df_tp[field_merge] = merge_val

df_agg_all_fields = pd.merge(df_agg_all_fields, df_tp, how = "outer").drop(field_merge, axis = 1)
fields_ord = [field_country, attr_tp.key ]
fields_dat = sorted([x for x in df_agg_all_fields.columns if (x not in fields_ord)])  
df_agg_all_fields = df_agg_all_fields[fields_ord + fields_dat]


df_agg_all_fields.to_csv(sa.fp_csv_soc_fields_by_country_simple, index = None, encoding = "UTF-8")